In [75]:
using HomotopyContinuation, HomologyInferenceWithWeakFeatureSize, DynamicPolynomials
hwfs = HomologyInferenceWithWeakFeatureSize
data_path = escape_string("paper_data/")
threshold = 1e-7

# Order of bottlenecks, change for other orders
k = 2

filename_root = "bots"*string(k)*"_"

# Uncomment if you have re-run the corresponding Bertini script
# data_path = escape_string("paper_data/bertini_inputs/"*str(k)*"_bottlenecks/")
# filename_root = ""

"bots2_"

In [76]:
num_vars = 3    
@polyvar q[1:num_vars]
x = q[1]
y = q[2]
z = q[3]
R = 3/2
r = 1
F = [(R^2 - r^2 + x^2 + y^2 + z^2)^2 - 4*R^2*(x^2 + y^2),x^3 + y^3 + z^3 + 1 - (x + y + z + 1)^3]
singular_points = hwfs.parse_bertini_file(data_path*filename_root*"singular_solutions")
singular_points = [hwfs.bertini_convert_solution_point_to_standard_form(point,num_vars,k) for point in singular_points]
nonsingular_points = hwfs.parse_bertini_file(data_path*filename_root*"nonsingular_solutions")
nonsingular_points = [hwfs.bertini_convert_solution_point_to_standard_form(point,num_vars,k) for point in nonsingular_points]

nondegenerate_nonsingular = [point for point in nonsingular_points if !hwfs.check_if_solution_is_degenerate(point;threshold=threshold)]
nondegenerate_singular = [point for point in singular_points if !hwfs.check_if_solution_is_degenerate(point;threshold=threshold)]

real_nonsingular = [point for point in nondegenerate_nonsingular if !(abs(imag(hwfs.apply_distance_squared_to_solution(point))) > threshold) ]
putative_real_nonsingular_contributors = [point for point in real_nonsingular if real(hwfs.apply_distance_squared_to_solution(point)) > threshold]
real_singular = [point for point in nondegenerate_singular if !(abs(imag(hwfs.apply_distance_squared_to_solution(point))) > threshold) ]
real_singular_contributors = [point for point in real_singular if real(hwfs.apply_distance_squared_to_solution(point)) > threshold]


distance_system_for_filtering,parameters_for_filtering = hwfs.minimum_distance_start_system(F,q)
start = randn(ComplexF64,num_vars+length(F))
initial_system = subs(distance_system_for_filtering,parameters_for_filtering => start)
initial_solution = solutions(solve(initial_system,start_system = :polyhedral,parameters=:conservative))
distances_nonsingular = [Inf]
if length(putative_real_nonsingular_contributors) > 0
    distances_nonsingular = [hwfs.filter_solution_to_bottleneck(solution,length(q),k,distance_system_for_filtering,start,parameters_for_filtering,initial_solution;threshold=threshold) for solution in putative_real_nonsingular_contributors]
    distances_nonsingular = [dist for dist in distances_nonsingular if dist!=false]
end
distances_singular = [Inf]
if length(real_singular_contributors) > 0 
    distances_singular = [sqrt(hwfs.apply_distance_squared_to_solution(point)) for point in real_singular_contributors]
end


Tracking 48 paths... 100%|████████████████████████████████████████| Time: 0:00:01
  # paths tracked:                  48
  # non-singular solutions (real):  40 (0)
  # singular solutions (real):      3 (0)
  # total solutions (real):         43 (0)


In [77]:
# This number is the total number of solutions computed on B_k(F)
length(singular_points)+length(nonsingular_points)

2736

In [78]:
# This counts the solutions which are contained in \Gamma_k(F)
length(singular_points) - length(nondegenerate_singular) + length(nonsingular_points) - length(nondegenerate_nonsingular)

576

In [79]:
# This counts the solutions which are not contained in \Gamma_k(F)
length(nondegenerate_singular)+length(nondegenerate_nonsingular)

2160

In [80]:
# The number of non-isolated solutions contributing 
# to the weak feature size
length(nondegenerate_singular)

0

In [81]:
# Computed algebraic k-bottlenecks
println(length(nondegenerate_nonsingular)/factorial(k))
# All non-degenerate non-singular with real distance
println(length(putative_real_nonsingular_contributors)/factorial(k))
# Real algebraic bottlenecks of V(F)
println(length([point for point in putative_real_nonsingular_contributors if hwfs.check_if_vector_is_real(point["circumcenter"];threshold=threshold)])/factorial(k))
# Real algebraic bottlenecks of V(F) \cap R^3
println(length([point for point in putative_real_nonsingular_contributors if hwfs.check_if_solution_has_real_endpoints(point;threshold=threshold)])/factorial(k))

1080.0
1080.0
68.0
50.0


In [71]:
# Critical values from non-isolated contributors
minimum(distances_singular)

Inf

In [72]:
# Critical values from isolated geometric k-bottlenecks
minimum(distances_nonsingular)

0.7720770556102096

In [82]:
length(distances_nonsingular)/factorial(k)

22.0